# Bright Stuff
### Health is our number 1 priority

In [ ]:
import pandas as pd
import numpy as np
import base64
import imageio as iio
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import os
import bar_chart_race as bcr
from IPython import display
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle
from plotting import Plotting
import plotly.graph_objects as go


In [ ]:
df = pd.read_pickle(f'site_2/site_2.pkl', compression='gzip') # We followed the helvar notebook closely

In [ ]:
# converting timestamp to appropriate format
df.loc[:, 'timestamp'] = (pd.to_datetime(df['timestamp'], utc=True)
                                 .dt.tz_convert('Europe/Helsinki')
                                 .dt.tz_localize(None))

In [ ]:
with open(f'site_2/site_2.png', "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

In [ ]:
img = iio.imread(f'site_2/site_2.png')
img.shape

In [ ]:
df_devices = pd.read_json(f'site_2/site_2.json')

In [ ]:
# choosing an exact day, because in our project we focus on the more detailed description of the day
df_events_day = df[df.timestamp.dt.date.astype(str) == "2021-09-15"].copy()

In [ ]:
cols = set(df_events_day['deviceid'])
df_devices = df_devices[df_devices['deviceid'].isin(cols)] # we want to know the individual devices

In [ ]:
# keeping sensors only in invastigated range (between zones) because we are only interested in the zones that look like offices
df_devices = df_devices[(df_devices['x'] > 333) & (df_devices['x'] < 4397)]
df_devices = df_devices[(df_devices['y'] > 1237) & (df_devices['y'] < 2625)]

df_devices['zone'] = 0

In [ ]:
# definition of zones
df_devices.loc[(df_devices['x'] > 333) & (df_devices['x'] < 1617), 'zone'] = 1
df_devices.loc[(df_devices['x'] > 1617) & (df_devices['x'] < 2565), 'zone'] = 2
df_devices.loc[(df_devices['x'] > 2565) & (df_devices['x'] < 3685), 'zone'] = 3
df_devices.loc[(df_devices['x'] > 3685) & (df_devices['x'] < 4397), 'zone'] = 4

In [ ]:
df_events_day = df_events_day[df_events_day['deviceid'].isin(df_devices['deviceid'])] #get those devices we need

df_events_day['zone'] = 0

In [ ]:
zone1 = list(df_devices.loc[(df_devices.zone == 1), 'deviceid'])
zone2 = list(df_devices.loc[(df_devices.zone == 2), 'deviceid'])
zone3 = list(df_devices.loc[(df_devices.zone == 3), 'deviceid'])
zone4 = list(df_devices.loc[(df_devices.zone == 4), 'deviceid'])

In [ ]:
df_events_day.loc[(df_events_day.deviceid.isin(zone1)), 'zone'] = 1
df_events_day.loc[(df_events_day.deviceid.isin(zone2)), 'zone'] = 2
df_events_day.loc[(df_events_day.deviceid.isin(zone3)), 'zone'] = 3
df_events_day.loc[(df_events_day.deviceid.isin(zone4)), 'zone'] = 4

In [ ]:
df_events_day.drop(columns=['deviceid'], inplace=True)

In [ ]:
df_events_day[df_events_day['zone'] == 4] #we have all the data summed for zone 4 for example

In [ ]:
df_events_day.timestamp = df_events_day.timestamp.dt.floor('10min')
df_events_day.loc[:, 'b'] = 1
df_events_day = df_events_day.groupby(['zone', 'timestamp']).sum()
df_events_day = df_events_day.pivot_table(index='timestamp', columns='zone', values='b')
df_events_day = df_events_day.reindex(pd.date_range(df_events_day.index.min().floor('1D'), df_events_day.index.max().ceil('1D'), freq='10min', closed='left')).fillna(0)

In [ ]:
df_events_day.drop(columns=[4], inplace=True)

In [ ]:
df_events_day # we needed to rotate the dataframe to have the rows as times

In [ ]:
# for the pitch we only took an interesting part of the day into account, but of course this can be done for other periods
df_shorter = df_events_day.loc[(df_events_day.index > '2021-09-15 05:20:00') & (df_events_day.index < '2021-09-15 08:40:00')]

In [ ]:
def plotting_thing(frame): # plotting function for the FuncAnimation package
    ax.clear()
    
    df = df_shorter
    min_val = 300 # somewhat arbitrary value should be tuned for a real situtation
    mid_val = 650
    
    # We highlight the performances of the departments with colours
    
    n1 = df[1][frame]
    if n1 <= min_val:
        c1 = 'r'
    elif n1 > min_val and n1 < mid_val:
        c1 = 'y'
    elif n1 >= mid_val:
        c1 = 'g'
    
    n2 = df[2][frame]
    if n2 <= min_val:
        c2 = 'r'
    elif n2 > min_val and n2 < mid_val:
        c2 = 'y'
    elif n2 >= mid_val:
        c2 = 'g'
        
    n3 = df[3][frame]    
    if n3 <= min_val:
        c3 = 'r'
    elif n3 > min_val and n3 < mid_val:
        c3 = 'y'
    elif n3 >= mid_val:
        c3 = 'g'
        
    plt.axis('off')
    ax.imshow(img)
    ax.add_patch(Rectangle((333, 2625), 1284, -1237, fill = True, lw = 1.5, alpha=0.5, fc=c1))
    ax.add_patch(Rectangle((1617, 2625), 948, -1237, fill = True, lw = 1.5, alpha=0.5, fc=c2))
    ax.add_patch(Rectangle((2565, 2625), 1120, -1237, fill = True, lw = 1.5, alpha=0.5, fc=c3))


In [ ]:
fig = plt.figure(figsize=(8, 6), dpi=100)
ax = fig.add_subplot(111)

anim = FuncAnimation(fig, plotting_thing, frames=19, interval=500) # animation 
#anim.save('faszfos.mp4', fps=2, extra_args=['-vcodec', 'libx264'])
'''
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()
'''

In [ ]:
# just for easier handling
df = df_events_day

In [ ]:
df.iloc[:, 0:] = df.iloc[:, 0:].cumsum() # creating a cummulative sum in time

In [ ]:
df = df.rename(columns = {1: 'Department 1', 2: 'Department 2', 3: 'Department 3'}) # just for looks
df

In [ ]:
df = df.loc[(df.index > '2021-09-15 05:20:00') & (df.index < '2021-09-15 08:40:00')]

In [ ]:
M = df.max().max() #some normalization so that we have at max 1000 movement points in the game

df = df/(M*0.001)
df

In [ ]:
#https://pypi.org/project/bar-chart-race/
bcr.bar_chart_race(df = df, n_bars = 3, sort='desc',title='COMPETITION',filename = 'compete.mp4')